In [ ]:
#Imports
import pathlib
import numpy as np
import pandas as pd

# pycytominer imports
from pycytominer.cyto_utils.cells import SingleCells
from pycytominer import aggregate, annotate, normalize, feature_select

import cytotable
# ignore mix type warnings from pandas
import warnings
#plotting
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle
import math
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import joypy
import scipy
from scipy import stats
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from sklearn.ensemble import IsolationForest

In [ ]:
# Setting file paths
data_dir = pathlib.Path("/mnt/bigdisk1/Allie_S/Replicative_Age_Project/CP_Output/Mar26").resolve(strict=True)
markers = "Mar26_mitolyso"
metadata_dir = (data_dir / "metadata").resolve(strict=True)

out_dir = pathlib.Path("/mnt/bigdisk1/Allie_S/Replicative_Age_Project/Data_Mining/test_output/results")
out_dir.mkdir(exist_ok=True)

# input file paths
plate_map = (metadata_dir / 'map.csv').resolve(strict=True)
image_path = pathlib.Path(data_dir / markers / "Image.csv").resolve(strict=True)
cell_path = pathlib.Path(data_dir / markers / "CellOutline.csv").resolve(strict=True)
nuc_path = pathlib.Path(data_dir / markers / "Nuclei.csv").resolve(strict=True)
lyso_path = pathlib.Path(data_dir / markers / "Lysosomes.csv").resolve(strict=True)
mito_path = pathlib.Path(data_dir / markers / "Mitochondria.csv").resolve(strict=True)

# setting output paths
sc_profiles_path = out_dir / "single_cell_profile.csv.gz"
anno_profiles_path = out_dir / "annotated_profile.csv.gz"
norm_profiles_path = out_dir / "normalized_profile.csv.gz"
feat_profiles_path = out_dir / "features_profile.csv.gz"


In [ ]:
# loading plate map and display it
platemap_df = pd.read_csv(plate_map)

# displaying platemap
print(platemap_df.columns.tolist())

image_df = pd.read_csv(image_path)
pre_nuclei_df = pd.read_csv(nuc_path, usecols=['ImageNumber','ObjectNumber','Metadata_Well', 'Metadata_Field', 
                                          'Metadata_WellColumn', 'Metadata_WellRow', 'AreaShape_Area',
                                         'AreaShape_Eccentricity','Parent_CellOutline'])
pre_cell_df = pd.read_csv(cell_path, usecols=['ImageNumber','ObjectNumber','Metadata_Well', 'Metadata_Field', 
                                          'Metadata_WellColumn', 'Metadata_WellRow', 'AreaShape_Area',
                                         'AreaShape_Eccentricity','Children_Lysosomes_Count', 'Children_Mitochondria_Count',
                                          'Math_Lysosome_CellOutline_Ratio', 'Math_Mitochondria_CellOutline_Ratio'])
pre_lyso_df = pd.read_csv(lyso_path, usecols=['ImageNumber','ObjectNumber','Metadata_Well', 'Metadata_Field',
                                          'Metadata_WellColumn', 'Metadata_WellRow', 'AreaShape_Area',
                                         'AreaShape_Eccentricity','Parent_CellOutline','Intensity_MedianIntensity_LAMP1',
                                         'Intensity_MeanIntensity_LAMP1','Texture_Contrast_LAMP1_3_01_256'])
pre_mito_df = pd.read_csv(mito_path, usecols=['ImageNumber','ObjectNumber','Metadata_Well', 'Metadata_Field',
                                          'Metadata_WellColumn', 'Metadata_WellRow', 'AreaShape_Area',
                                         'AreaShape_Eccentricity','Parent_CellOutline','Intensity_MedianIntensity_MitoTracker',
                                         'Intensity_MeanIntensity_MitoTracker','Texture_Contrast_MitoTracker_3_01_256'])
#This will definitley crash... cannot merge organelle-per-cell realistically
##merged_data = cell_df.merge(lyso_df, how ='left', left_on='ObjectNumber', right_on='Parent_CellOutline')


In [ ]:
#Merge the dfs with the metadata
cell_df = pre_cell_df.merge(platemap_df, on=['Metadata_Well','Metadata_WellRow','Metadata_WellColumn','Metadata_Field'], how ='left')
lyso_df = pre_lyso_df.merge(platemap_df, on=['Metadata_Well','Metadata_WellRow','Metadata_WellColumn','Metadata_Field'], how ='left')
mito_df = pre_mito_df.merge(platemap_df, on=['Metadata_Well','Metadata_WellRow','Metadata_WellColumn','Metadata_Field'], how ='left')
nuclei_df = pre_nuclei_df.merge(platemap_df, on=['Metadata_Well','Metadata_WellRow','Metadata_WellColumn','Metadata_Field'], how ='left')


#print(cell_df.head(100))


In [ ]:
#Features to be used in plots and stats
cell_features = ['AreaShape_Area','AreaShape_Eccentricity','Children_Lysosomes_Count', 'Children_Mitochondria_Count',
                 'Math_Lysosome_CellOutline_Ratio', 'Math_Mitochondria_CellOutline_Ratio']
lyso_features = ['AreaShape_Area','AreaShape_Eccentricity','Intensity_MedianIntensity_LAMP1',
                 'Intensity_MeanIntensity_LAMP1','Texture_Contrast_LAMP1_3_01_256']
mito_features = ['AreaShape_Area','AreaShape_Eccentricity','Intensity_MedianIntensity_MitoTracker',
                 'Intensity_MeanIntensity_MitoTracker','Texture_Contrast_MitoTracker_3_01_256']
nuclei_features = ['AreaShape_Area','AreaShape_Eccentricity']

In [ ]:
#Make time int
for df in [mito_df, nuclei_df, cell_df, lyso_df]:

    df['Time'] = df['Time'].astype(int)

def ratioCalc(df1, df2):
  int1 = df1['Intensity_MedianIntensity_CompensatedTfn']
  int2 = df2['Intensity_MedianIntensity_CompensatedTfn']

  temp_copy1 = outlier_removal(df1, int1)
  temp_copy2 = outlier_removal(df2, int2)

  intensity_ratio = temp_copy1[int1] / temp_copy1[int2]
  return df[intensity_ratio]

def normalization(df, column): #normalizing intensity from 0 to 1
  min_intensity = df[column].min()
  max_intensity = df[column].max()
  df[column] = (df[column] - min_intensity) / (max_intensity - min_intensity)
  return df[column]

def mad_normalization(df, column):

  mad = scipy.stats.median_abs_deviation(df[column])
  outlier_mask = np.abs(df[column] - df[column].median()) > mad*1.5
  cleaned_df = df[~outlier_mask]
  #recalaculate mad and make a mask of mads that go over the threshold (then negate them)
  return cleaned_df[column]

def outlier_removal(df, column):
    # Create a copy of the column and the 'Time' column

    if 'Ikarugamycin' in df: #remove this for now
      filtered_df = df[df['Drug'] != 'Ikarugamycin']

      column_copy = filtered_df[column].copy()
      time_column = filtered_df['Time'].copy()
      parent_column = filtered_df['Parent_Nuclei'].copy()

      mini_df = pd.DataFrame({column: column_copy,'Time': time_column,'Parent_Nuclei' : parent_column})
      mini_df = mini_df.dropna()
    else:
      column_copy = df[column].copy()
      time_column = df['Time'].copy()
       # Create a mini DataFrame with the column and 'Time' values
      mini_df = pd.DataFrame({column: column_copy, 'Time': time_column})
      mini_df = mini_df.dropna()

    #remove values for each type of data
    if df.equals(nuclei_df):
      # remove stuff within the range of t4
      four_mean = np.mean(column_copy[df['Time'] == 4])
      std_dev = np.std(column_copy[df['Time'] == 4])
      threshold = four_mean + (3 * std_dev)

      mini_df = mini_df[mini_df[column] <= threshold]
      mini_df = mini_df.reset_index(drop=True)


    if 'AreaShape' in column:
      mini_df[column] = mini_df[column].astype(float)
      mini_df = mini_df.loc[(mini_df[column] <= 300000) & (mini_df[column] > 0)]
      mini_df = mini_df.reset_index(drop=True)

    if 'Texture' in column:
      mini_df[column] = mini_df[column].astype(float)
      mini_df = mini_df.loc[(mini_df[column] <= 200) & (mini_df[column] > 0)]
      mini_df = mini_df.reset_index(drop=True)

      # Calculate top and bottom percentiles for Time == 0
    p1 = np.percentile(mini_df[column], 5)
    p3 = np.percentile(mini_df[column], 95)

    # Filter out values greater than 3IQR from Q1 or Q3
    filtered_mini_df = mini_df.loc[(mini_df[column] >= p1) & (mini_df[column] <= p3)]
    #Train outlier detection algorithm on non-images csvs
    if df.dropna().equals(image_df.dropna()):
    #pd.testing.assert_frame_equal(df,image_df):
      return filtered_mini_df
    else:
      cleaned_df = []
      groups = filtered_mini_df['Time'].unique()
      for group in groups:

        group = int(group)
        group_mini_df = filtered_mini_df[filtered_mini_df['Time'] == group]

        X_1D = group_mini_df[column].values
        X = X_1D.reshape(-1, 1)
        clf = IsolationForest(n_estimators=20, random_state=42, contamination='auto')
        clf.fit(X)  # fit 20 trees

        #Predict outliers and remove from X
        preds = clf.predict(X)
        outlier_indices = np.where(preds == -1)[0]
        # Create a masked df for each group to block out the values of outliers from the original dataset (use ~ to negate outliers so they can be removed)
        cleaned_group_df = group_mini_df.loc[~group_mini_df.index.isin(outlier_indices)]
        #add the masked df to the list
        cleaned_df.append(cleaned_group_df)

      cleaned_df = pd.concat(cleaned_df, ignore_index=True) #collapse list into df
    return pd.DataFrame(cleaned_df)



# if df.equals(mito_df) or df.equals(lysosomes_df):
# if df[Parents] == nuclei_df[ObjectNumber]. - only have the above if
#if 'Area' in column:
# Calculate mean and standard deviation for Time == 4

In [ ]:
def normalization(df, column): #normalizing intensity from 0 to 1
  min_intensity = df[column].min()
  max_intensity = df[column].max()
  df[column] = (df[column] - min_intensity) / (max_intensity - min_intensity)
  return df[column]

def z_normalization(df, column):

  df[column] = df[column] - df[column].mean() / df[column].std()
  return df[column]

#median absolute deviation; normalize using median version of standard deviation
def mad_normalization(df, column):

  mad = scipy.stats.median_abs_deviation(df[column])
  outlier_mask = np.abs(df[column] - df[column].median()) > mad*1.5
  cleaned_df = df[~outlier_mask]
  #recalaculate mad and make a mask of mads that go over the threshold (then negate them)
  return cleaned_df[column]


In [ ]:
stats_cols = ['AreaShape_Area','AreaShape_ConvexArea','Intensity_MedianIntensity_CompensatedTfn','Intensity_MeanIntensity_CompensatedTfn']


def stats(df, cols, excel_name):
    with pd.ExcelWriter(excel_name) as writer:

      for column in cols:
        temp_copy = df.copy()  # Create a copy of the DataFrame for processing

        if 'Intensity' in column:
          temp_copy[column] = normalization(temp_copy, column)
          temp_copy = outlier_removal(temp_copy, column)
        else:
          temp_copy[column] = mad_normalization(temp_copy, column)

          temp_copy = outlier_removal(temp_copy, column)

        # Perform pairwise Tukey's HSD test
        tukey = pairwise_tukeyhsd(endog=temp_copy[column], groups=temp_copy['Time'])

        # Extract relevant results
        results = np.array(tukey.summary().data)[:, [0, 1, 3, 6]]
        df_results = pd.DataFrame(results, columns=['Group 1', 'Group 2', 'p-value', 'Reject']).drop([0])
        df_results.reset_index(drop=True, inplace=True)
        df_results[['Group 1', 'Group 2']] = df_results[['Group 1', 'Group 2']].astype(int)
        df_results['p-value'] = df_results['p-value'].astype(float)
        # Truncate the column name if it exceeds 31 characters
        name = column[:31]

        # Save DataFrame to Excel sheet without the index column
        df_results.to_excel(writer, sheet_name=name, index=False)


# Call your stats below

# Make the plots and validate dist

In [ ]:
def make_layout(xtitle,ytitle):
  design = go.Layout(
        plot_bgcolor="#FFF",
        xaxis=dict(
            title=xtitle,
            linecolor="black",
            showgrid=False,
            titlefont=dict(size=20),
            tickfont=dict(size=16, color="black")
        ),
        yaxis=dict(
            title=ytitle,
            linecolor="black",
            showgrid=False,
            titlefont=dict(size=20),
            tickfont=dict(size=16, color="black")
        ),
        font=dict(size=14),
        legend=dict(
            title="",
            itemsizing='constant',
            font=dict(size=16, color="black"),
            tracegroupgap=10,
            traceorder='normal',
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        boxgap=0.4,
        boxgroupgap=0.05,
        width=900,  # Specify width of the plot
        height=600
    )
  return design

def box_param(fig, color, design):
  fig.update_traces(boxmean=True)
  fig.update_traces(jitter=1.0)
  fig.update_traces(boxpoints=False)
  fig.update_layout(design)
  fig.update_traces({'opacity': 0.9})
  fig.update_traces(marker_color=color)
  return fig

In [ ]:
def boxplot(df, variable, ytitle, xtitle, box_color, save=False, save_name=None):
    # #some styling stuff
  layout = make_layout(xtitle,ytitle)
  temp_copy = df.copy()
  if 'Intensity' in variable:
    temp_copy = outlier_removal(temp_copy, variable)
    temp_copy[variable] = normalization(temp_copy, variable)

  else:
    temp_copy = outlier_removal(temp_copy, variable)
    temp_copy[variable] = mad_normalization(temp_copy, variable)

  fig = px.box(temp_copy, x="Time", y=variable)
  fig = box_param(fig, box_color, layout)
  fig.update_traces(quartilemethod="linear")
  
  if save==True:
      fig.write_image(save_name)

##  Call plotly
```python
boxplot(df, # dataframe name: mito_df, nuclei_df, image_df, outline_df, lysosomes_df
        'Variable', # the variable you wanna plot, column name
        'Y Title',# name of your y-axis (custom)
        'Time Point',#name of your x-axis (custom)
        'Red', # color of the boxes
        save=True, # if wanna save change to False to True, False is default
        save_name='_boxplot.png') #specify the name of the plot that you save

```

In [ ]:
#save all the files for that one feature

colors_i = 0

for feature in cell_features:
    title = 'Cell_' + feature
    boxplot(cell_df,
            feature,
            feature,
            "Time Point",
            px.colors.qualitative.Pastel[colors_i],
            save = True,
            save_name = feature + "_boxplot.png")
    colors_i = colors_i + 1
    
            
    

In [ ]:
fig = px.histogram(lyso_df, x='Intensity_MedianIntensity_LAMP1', color = 'Time')
fig.show()

In [ ]:
fig = px.histogram(outlier_removal(lyso_df,'Intensity_MedianIntensity_LAMP1'), x='Intensity_MedianIntensity_LAMP1', color = 'Time')
fig.show()

# Pycytominer Testing: Use on single-cell profiles

In [ ]:
# annotating merged single-cell profile with metadata
annotate(
    profiles=sc_profiles_path,
    platemap=platemap_df,
    join_on=["Metadata_well", "Image_Metadata_Well"],
    output_file=anno_profiles_path,
    compression_options="gzip",
)

# save message display
print(f"Annotated profile saved in: {anno_profiles_path}")


In [ ]:
# normalize dataset
normalize(
    profiles=anno_profiles_path,
    features="infer",
    image_features=False,
    meta_features="infer",
    samples="all",
    method="standardize",
    output_file=norm_profiles_path,
    compression_options="gzip",
)

# save message display
print(f"Normalized profile saved in: {norm_profiles_path}")


In [ ]:
# creating selected features profile
feature_select(
    profiles=norm_profiles_path,
    features="infer",
    image_features=False,
    samples="all",
    operation=["variance_threshold", "correlation_threshold", "blocklist"],
    output_file=feat_profiles_path,
    compression_options="gzip",
)

# save message display
print(f"Selected features profile saved in: {feat_profiles_path}")
